# 第五次作业

为能够在notebook里面正确运行并显示，额外需要ffmpeg环境。

### 1、 一阶对流方程

对于此方程，求解器如下：

In [1]:
import numpy as np
from matplotlib import pyplot as plt
from math import floor, exp, pi
from matplotlib import rc, animation

rc('animation', html='html5')

def finit(x):
    if x > -2:
        return 1
    else:
        return 0

# a=-1,x=-15:1,t=0:4
def solvepde(dx, dt):
    x0, xn, t0, tn = -15., 1., 0., 4
    nx = floor((xn - x0) / dx) + 1
    nt = floor((tn - t0) / dt) + 1
    sol = np.zeros((nt, nx))  # sol[t,x]
    xs = np.array([x0 + dx * i for i in range(0, nx)])
    # sol[0, :] = np.vectorize(finit)(np.array([x0 + dx * i for i in range(0, nx)]))
    sol[0, :] = np.vectorize(finit)(xs)
    for i in range(1, nt):
        sol[i, nx - 1] = 0  # 边界
        for k in range(0, nx - 1):
            sol[i, k] = sol[i - 1, k] + dt / dx * (sol[i - 1, k + 1] - sol[i - 1, k])
    return xs, sol



dx=0.05,dt=0.04:

In [2]:
xs, sol = solvepde(0.05, 0.04)
nt = floor(4 / 0.04)
fig = plt.figure()
plt.xlim(-15,1)
plt.ylim(0, 1.2)
plt.grid()
ims = []
for i in np.arange(nt):
    ims.append(plt.plot(xs, sol[i, :], 'r-'))

im_ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True)

im_ani

dx=0.05,dt=0.05:

In [3]:
xs, sol = solvepde(0.05, 0.05)
nt = floor(4 / 0.05)
fig = plt.figure()
plt.xlim(-15,1)
plt.ylim(0, 1.2)
plt.grid()
ims = []
for i in np.arange(nt):
    ims.append(plt.plot(xs, sol[i, :], 'r-'))

im_ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True)

im_ani

dx=0.05,dt=0.06:

In [4]:
xs, sol = solvepde(0.05, 0.06)
nt = floor(4 / 0.06)
fig = plt.figure()
plt.xlim(-15,1)
plt.ylim(0, 1.2)
plt.grid()
ims = []
for i in np.arange(nt):
    ims.append(plt.plot(xs, sol[i, :], 'r-'))

im_ani = animation.ArtistAnimation(fig, ims, interval=50, blit=True)

im_ani

### 2、 Kruskal, Zalusky孤立子

使用了numba的jit，求解器的代码如下：

In [5]:
from numba import jit


# 一阶差分每一步的步骤
@jit
def firststep(u1, dx, dt, e, d):
    n = np.shape(u1)[0]
    u = np.zeros(n)
    for j in range(0, n):
        u[j] = u1[j] - (u1[(j + 1) % n] + u1[j % n] + u1[(j - 1) % n]) * (
            u1[(j + 1) % n] - u1[(j - 1) % n]) * dt / dx * e / 6 - (u1[(j + 2) % n] + 2 * u1[(j - 1) % n] - 2 * u1[
            (j + 1) % n] - u1[(j - 2) % n]) * dt / dx / dx / dx * d ** 2 / 2
    return u


# 中心差分每一步的步骤
@jit
def nextstep(u1, u2, dx, dt, e, d):
    n = np.shape(u1)[0]
    u3 = np.zeros(n)
    for j in range(0, n):
        u3[j] = u1[j] - (u2[(j + 1) % n] + u2[j % n] + u2[(j - 1) % n]) * (
            u2[(j + 1) % n] - u2[(j - 1) % n]) * dt / dx * e / 3 - (u2[(j + 2) % n] + 2 * u2[(j - 1) % n] - 2 * u2[
            (j + 1) % n] - u2[(j - 2) % n]) * dt / dx / dx / dx * d ** 2
    return u3


@jit
def sov1(nx, dt, nt, e, d):
    xs = np.linspace(0, 2, nx)
    dx = xs[1] - xs[0]
    u = np.zeros((nt, nx))
    u[0, :] = np.cos(pi * xs)
    for i in range(0, nt - 1):
        u[i + 1, :] = firststep(u[i, :], dx, dt, e, d)
    return xs, u


@jit
def sov2(nx, dt, nt, e, d):
    xs = np.linspace(0, 2, nx)
    dx = xs[1] - xs[0]
    u = np.zeros((nt, nx))
    u[0, :] = np.cos(pi * xs)
    u[1, :] = firststep(u[0, :], dx, dt, e, d)
    for i in range(1, nt - 1):
        u[i + 1, :] = nextstep(u[i - 1, :], u[i, :], dx, dt, e, d)
    return xs, u


# 一阶差分求解器
# 对时间t，总步数为nt*dnt个，隔dnt次迭代记录一次数据，共记录nt组数据
@jit
def solve1(nx, dt, nt, dnt, e, d):
    xs = np.linspace(0, 2, nx)
    dx = xs[1] - xs[0]
    u = np.zeros((nt, nx))
    u[0, :] = np.cos(pi * xs)
    ut = np.copy(u[0, :])
    for i in range(0, nt - 1):
        for j in range(0, dnt):
            ut = firststep(ut, dx, dt, e, d)
        u[i + 1, :] = ut
    return xs, u


# 中心差分求解器
@jit
def solve2(nx, dt, nt, dnt, e, d):
    xs = np.linspace(0, 2, nx)
    dx = xs[1] - xs[0]
    u = np.zeros((nt, nx))
    u[0, :] = np.cos(pi * xs)
    ut1 = np.copy(u[0, :])
    ut2 = firststep(ut1, dx, dt, e, d)
    for i in range(0, nt - 1):
        for j in range(0, dnt):
            ut2, ut1 = nextstep(ut1, ut2, dx, dt, e, d), ut2
        u[i + 1, :] = ut1
    return xs, u

虽然同时给出一阶差分和中心差分的算法，这里使用一阶差分进行计算，二阶差分极不稳定，在曲线还是光滑的情况下会突然崩溃。这里参数取$dx=2/(150-1)=0.013$，$dt=0.0000025$，在时间上计算一千万步，取其中的1000个点进行存储。

In [6]:
fig = plt.figure()
ax = plt.axes(xlim=(0, 2), ylim=(-2, 5))
line, = ax.plot([], [], lw=2)
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
xs, sol = solve1(150, 0.0000025, 1000, 10000, 1, 0.022)
print('solving done')


def fig_ini():
    line.set_data([], [])
    time_text.set_text('')
    return line, time_text


def animate(i):
    line.set_data(xs, sol[i,:])
    time_text.set_text('t = %f' % (i*0.025))
    return line, time_text


anim = animation.FuncAnimation(fig, animate, init_func=fig_ini,
                               frames=1000, interval=40, blit=True)

anim

solving done
